# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

**Note: The following line of code has to be adapted to follow your folder structure.**

In [ ]:
env = UnityEnvironment(file_name='resources/environments/Reacher_Windows_x86_64_twenty/Reacher.exe')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
import numpy as np

env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to the environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

### 4. Let's train an agent learning from 20 independent environments (in parallel)

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import torch

from src.model import Actor
from src.model import Critic
from src.ddpg_agent import OUNoise
from src.replay_buffer import ReplayBuffer


BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64        # minibatch size
random_seed = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Actor Network (w/ Target Network)
actor_local = Actor(state_size, action_size, random_seed).to(device)
actor_target = Actor(state_size, action_size, random_seed).to(device)

# Critic Network (w/ Target Network)
critic_local = Critic(state_size+action_size, 1, random_seed).to(device)
critic_target = Critic(state_size+action_size, 1, random_seed).to(device)

# Noise process
noise_process = OUNoise(action_size, random_seed)

# Replay memory
memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE, random_seed, device)

In [ ]:
from src.ddpg_agent import Agent
from collections import deque

max_episodes = 300
max_timesteps = 1000

model_dir = 'models'
checkpoint_actor = 'checkpoint_actor.pth'
checkpoint_critic = 'checkpoint_critic.pth'
model_actor = 'model_actor.pt'
model_critic = 'model_critic.pt'


def ddpg(n_episodes=max_episodes, n_timesteps=max_timesteps):
    """Deep Deterministic Policy Gradient.

    Args:
        n_episodes (int): maximum number of training episodes
        n_timesteps (int): maximum number of timesteps per episode
    """
    mean_scores = []  # list containing average scores from each episode
    mean_scores_window = deque(maxlen=100)  # last 100 average scores

    agent = Agent(actor_local, actor_target, critic_local, critic_target, noise_process, memory)

    for episode in range(1, n_episodes + 1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        # initialize the mean score (averaged over each agent)
        scores = np.zeros(num_agents)
        mean_score = 0
        for timestep in range(n_timesteps):
            actions = agent.act(states)
            actions = np.clip(actions, -1, 1)
            # send all actions to the environment
            env_info = env.step(actions)[brain_name]
            # get next state (for each agent)
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            for i in range(num_agents):
                agent.step(states[i], actions[i], rewards[i], next_states[i], dones[i])
            agent.learn()
            states = next_states
            scores += rewards
            if timestep % 10 == 0:
                mean_score = np.mean(scores)
                std_score = np.std(scores)
                print('\rTimestep {}\tMean Score: {:.2f}\tStd Score: {:.2f}'.
                      format(timestep, mean_score, std_score), end="")
            if np.any(dones):
                break

        mean_scores_window.append(mean_score)  # save most recent score
        mean_scores.append(mean_score)  # save most recent score

        if episode % 1 == 0:
            print(
                '\nEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(mean_scores_window)))
        if np.mean(mean_scores_window) >= 30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(
                episode - 100, np.mean(mean_scores_window)))
            agent.actor_local.save_checkpoint(model_dir + '/' + checkpoint_actor)
            agent.actor_local.save_model(model_dir + '/' + model_actor)
            agent.critic_local.save_checkpoint(model_dir + '/' + checkpoint_critic)
            agent.critic_local.save_model(model_dir + '/' + model_critic)
            break
    return mean_scores


avg_scores = ddpg()

In [ ]:
import matplotlib.pyplot as plt

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(avg_scores)), avg_scores)
plt.ylabel('Average Score')
plt.xlabel('Episode #')
plt.title('DDPG Agent results: Average Scores vs. Episode #')
plt.show()

### 5. Let's watch our trained Agent

In [ ]:
# load the weights from file
from src.ddpg_agent import Agent
from src.model import load_actor
from src.model import load_critic

agent = Agent(actor_local, actor_target, critic_local, critic_target, noise_process, memory)
agent.actor_local = load_actor(model_dir + '/' + model_actor)
agent.critic_local = load_critic(model_dir + '/' + model_critic)

for i_episode in range(1):
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations
    scores = 0
    while True:
        actions = agent.act(states)
        actions = np.clip(actions, -1, 1)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        states = next_states
        scores += np.mean(rewards)
        if any(dones):
            break
    print("Episode: {}, Score: {}".format(i_episode, scores))

In [ ]:
env.close()